In [27]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
!pip install pymorphy2
import pymorphy2
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
import copy
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import metrics

In [2]:
train_data = "/kaggle/input/nlp-data/train_data_exam_1.xlsx"
df_train = pd.read_excel(train_data, 0)  
test_data = "/kaggle/input/nlp-data/test_data_exam_1.xlsx"
df_test = pd.read_excel(test_data, 0)  

# this gives us the size of the array
print('Dataset size', df_train.shape)

# here we can get size of array as two variables
num_rows, num_feature = df_train.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(df_train))

print('-------------------')
print('full data loaded')
print('-------------------')
print(df_train)

Dataset size (18779, 2)
row number:  18779
feature number:  2

names of features:  ['original_data', 'score']
-------------------
full data loaded
-------------------
                                           original_data  score
0      Вот тут у Галы:http://vaskodagala.livejournal....      0
1      Мужик, разомлевший то ли от коньяка, то ли от ...      0
2      ро треть бедных - тоже вранье. Даже если говор...     -1
3      все относительно.  законодатели достали конкре...     -1
4      А Алтайский край и Республика Алтай - разные с...      0
...                                                  ...    ...
18774  да агония это уже  Аххахахахахахахааа!!!!!  Пи...     -2
18775  Для колониальных войн/войн за ресурсы, ессно! ...     -2
18776  Как в одном из наших роликов, когда боевиков о...     -1
18777  Ты знаешь, да, идея, что у нас такая среда, чт...     -1
18778   жалость -  Ой, бедный ты. На тебе конфетку. И...      0

[18779 rows x 2 columns]


In [3]:
def clear_and_lematization(df):
    morph = pymorphy2.MorphAnalyzer()
    for i in tqdm(range(len(df))):
        s = str(df['original_data'][i])  
        s=re.sub(r'http\S+', '', s) 
        s = s.lower()
        s="".join(c for c in s if c.isalpha() or c ==" ")
        s = re.sub(r'\s+', ' ', s)
        
        # how we are doing lematization
        tokens = word_tokenize(s)
        doc_stem=''
        for j in range(0,len(tokens)):
            tokens[j] = morph.parse(tokens[j])[0].normal_form
            doc_stem = doc_stem + tokens[j] + ' '

        df['original_data'][i]=doc_stem
    return df
df_train=clear_and_lematization(df_train)
print(df_train)

  0%|          | 0/18779 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
100%|██████████| 18779/18779 [13:07<00:00, 23.84it/s]

                                           original_data  score
0      вот тут у гала небольшой коментарий от pocahon...      0
1      мужик разомлеть то ли от коньяк то ли от мой щ...      0
2      ро треть бедный тоже враньё даже если говорить...     -1
3      всё относительно законодатель достать конкретн...     -1
4      а алтайский край и республика алтай разный суб...      0
...                                                  ...    ...
18774  да агония это уже аххахахахахахахааа пиздец до...     -2
18775  для колониальный войнвойна за ресурс ессный ил...     -2
18776  как в один из наш ролик когда боевик окружить ...     -1
18777  ты знать да идея что у мы такой среда что надо...     -1
18778  жалость ой бедный ты на ты конфетка или хоть п...      0

[18779 rows x 2 columns]


In [4]:
df_test=clear_and_lematization(df_test)

  0%|          | 0/8048 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
100%|██████████| 8048/8048 [05:26<00:00, 24.62it/s]


In [6]:
def stop_words(df):
    my_stop = set(stopwords.words('russian'))
    for i in tqdm(range(len(df))):
        s = str(df['original_data'][i])  
        tokens = word_tokenize(s)
        tokens_without_sw = [word for word in tokens if not word in my_stop]
        df['original_data'][i]=" ".join(tokens_without_sw)
    return df
df_train=stop_words(df_train)
df_test=stop_words(df_test)
print(df_train)

  0%|          | 0/18779 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|██████████| 8048/8048 [00:09<00:00, 843.63it/s]

                                           original_data  score
0      гала небольшой коментарий pocahontasm рекоменд...      0
1      мужик разомлеть коньяк щастие согласиться пост...      0
2      ро треть бедный враньё говорить столица провин...     -1
3      всё относительно законодатель достать конкретн...     -1
4      алтайский край республика алтай разный субъект...      0
...                                                  ...    ...
18774  агония это аххахахахахахахааа пиздец долбоеб и...     -2
18775  колониальный войнвойна ресурс ессный всякий ку...     -2
18776  наш ролик боевик окружить возможность сопротив...     -1
18777  знать идея среда снижать риск путём невысовыва...     -1
18778  жалость ой бедный конфетка поплакать вместе вз...      0

[18779 rows x 2 columns]


In [11]:
df=copy.deepcopy(df_train)
print(df)

                                           original_data  score
0      гала небольшой коментарий pocahontasm рекоменд...      0
1      мужик разомлеть коньяк щастие согласиться пост...      0
2      ро треть бедный враньё говорить столица провин...     -1
3      всё относительно законодатель достать конкретн...     -1
4      алтайский край республика алтай разный субъект...      0
...                                                  ...    ...
18774  агония это аххахахахахахахааа пиздец долбоеб и...     -2
18775  колониальный войнвойна ресурс ессный всякий ку...     -2
18776  наш ролик боевик окружить возможность сопротив...     -1
18777  знать идея среда снижать риск путём невысовыва...     -1
18778  жалость ой бедный конфетка поплакать вместе вз...      0

[18779 rows x 2 columns]


In [13]:
def vectorization(df):
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(df['original_data']))]
    #We have successfully cleaned the documents and let’s create the model

    d2v_model = Doc2Vec(documents, vector_size=300, epochs=8, workers=4)
    #w2v_model = Word2Vec(min_count=10, window=2, vector_size=300, negative=10, alpha=0.03, min_alpha=0.0007, sample=6e-5, sg=1)

    Y = df['score'].values
    encoded_docs = [d2v_model.infer_vector([x]) for x in df["original_data"]]

    df["original_data"] = encoded_docs
    return df

In [14]:
df=vectorization(df)

In [15]:
df

,original_data,score
0,"[-0.00038517645, 0.00088026223, -0.000894949, ...",0
1,"[0.0014365711, -9.28396e-05, -0.000854706, -0....",0
2,"[-0.0013412577, -0.0015426081, -0.00018100967,...",-1
3,"[-0.00068813405, -0.00029315392, -8.4696214e-0...",-1
4,"[0.0014163087, 0.0012789526, -0.00045867762, 0...",0
...,...,...
18774,"[0.0005942329, 3.3465625e-05, -0.0011421473, -...",-2
18775,"[-0.0012819793, 0.00033048927, 0.0006386216, -...",-2
18776,"[-0.0003628096, -0.00013684412, -0.00019942621...",-1
18777,"[0.0006082211, -0.0006100209, 0.00021320283, 0...",-1


In [16]:
df_test=vectorization(df_test)

In [17]:
print(df_test)

                                          original_data  score
0     [-0.0007005358, -0.0010926597, -0.0008658411, ...      0
1     [0.00050507725, 0.0015609633, -0.00063179614, ...      0
2     [0.0012362582, -0.0012174068, 0.001658022, -0....      1
3     [-0.0006065422, 0.0012920747, 0.0011434853, 0....     -1
4     [-3.506144e-06, -0.0011744092, 0.0005202212, 0...     -1
...                                                 ...    ...
8043  [0.00033375024, -4.824559e-05, 0.0011086279, 0...      0
8044  [-0.0002488044, -0.0012059625, -0.0004676775, ...      0
8045  [-0.00030789195, 0.0008987969, 0.0014729772, -...      0
8046  [-0.0011191082, 0.0011925958, -0.0012218261, 0...      0
8047  [-0.0016263147, -0.0007113282, -0.001607763, 0...      0

[8048 rows x 2 columns]


In [18]:
df["score"].value_counts()

 0    9795
-1    6468
 1    1206
-2    1060
 2     250
Name: score, dtype: int64

In [19]:
from sklearn.utils import resample
df0=df[df["score"]==0]
df1=df[df["score"]==1]
df1n=df[df["score"]==-1]
df2=df[df["score"]==2]
df2n=df[df["score"]==-2]

#df0_down=resample(df0, replace=True, n_samples=5000)
df0_down=df0
df1_up=resample(df1, replace=True, n_samples=5000)
#df1n_down=resample(df1n, replace=True, n_samples=5000)
df1n_down=df1n
df2_up=resample(df2, replace=True, n_samples=5000)
df2n_up=resample(df2n, replace=True, n_samples=5000)

df=pd.concat([df0_down,df1_up,df1n_down, df2_up, df2n_up])
df["score"].value_counts()


 0    9795
-1    6468
 1    5000
 2    5000
-2    5000
Name: score, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['original_data'], df['score'], test_size=0.2, random_state=42)
X_train=np.stack(X_train)
X_test=np.stack(X_test)
X_train.shape

(25010, 300)

## kNN

На тренировочных данных:

In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', leaf_size=1, p=2)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)
predicted = knn.predict(X_test)

print('f1_score: ', f1_score(y_test, predicted, average="macro"))
print('precision_score: ', precision_score(y_test, predicted, average="macro"))
print('recall_score', recall_score(y_test, predicted, average="macro")) 

print(metrics.classification_report(y_test, predicted))

f1_score:  0.7005882639860772
precision_score:  0.6914321736624227
recall_score 0.7116985313264393
              precision    recall  f1-score   support

          -2       0.79      0.87      0.83       987
          -1       0.41      0.39      0.40      1278
           0       0.59      0.54      0.56      1969
           1       0.77      0.83      0.80      1023
           2       0.91      0.92      0.92       996

    accuracy                           0.67      6253
   macro avg       0.69      0.71      0.70      6253
weighted avg       0.66      0.67      0.67      6253



На тестовых

In [32]:
predicted = knn.predict(np.stack(df_test["original_data"]))
print(metrics.classification_report(df_test["score"], predicted))

              precision    recall  f1-score   support

          -2       0.48      0.63      0.55       473
          -1       0.66      0.65      0.66      2747
           0       0.78      0.72      0.75      4199
           1       0.50      0.66      0.57       524
           2       0.43      0.73      0.54       105

    accuracy                           0.69      8048
   macro avg       0.57      0.68      0.61      8048
weighted avg       0.70      0.69      0.69      8048



## Random forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,min_samples_leaf=1,max_depth=None)
clf.fit(X_train, y_train)

RandomForestClassifier()

Результат на тренировочных данных:

In [28]:
predicted = clf.predict(X_test)

print('f1_score: ', f1_score(y_test, predicted, average="macro"))
print('precision_score: ', precision_score(y_test, predicted, average="macro"))
print('recall_score', recall_score(y_test, predicted, average="macro")) 

# calculating the main quality metrics
print(metrics.classification_report(y_test, predicted))

f1_score:  0.7554768186518996
precision_score:  0.757249153495004
recall_score 0.7710483058342994
              precision    recall  f1-score   support

          -2       0.88      0.95      0.92       987
          -1       0.46      0.24      0.32      1278
           0       0.62      0.76      0.69      1969
           1       0.88      0.90      0.89      1023
           2       0.94      1.00      0.97       996

    accuracy                           0.75      6253
   macro avg       0.76      0.77      0.76      6253
weighted avg       0.72      0.75      0.73      6253



На тестовых данных:

In [29]:
X=np.stack(df_test['original_data'])
y=df_test['score']
predicted = clf.predict(X)

print('f1_score: ', f1_score(y, predicted, average="macro"))
print('precision_score: ', precision_score(y, predicted, average="macro"))
print('recall_score', recall_score(y, predicted, average="macro")) 

# calculating the main quality metrics
print(metrics.classification_report(y, predicted))

f1_score:  0.6609894021464616
precision_score:  0.6326943215181949
recall_score 0.7118239206636917
              precision    recall  f1-score   support

          -2       0.57      0.68      0.62       473
          -1       0.76      0.58      0.66      2747
           0       0.75      0.82      0.78      4199
           1       0.58      0.68      0.63       524
           2       0.50      0.79      0.61       105

    accuracy                           0.72      8048
   macro avg       0.63      0.71      0.66      8048
weighted avg       0.73      0.72      0.72      8048



## XGBoost

In [33]:
import xgboost as xgb
model = xgb.XGBClassifier(learning_rate=0.1,n_estimators=100)
y_train_pos=y_train+2
model.fit(X_train, y_train_pos)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

На тренировочных данных:

In [34]:
predicted = model.predict(X_test)
predicted-=2
print('f1_score: ', f1_score(y_test, predicted, average="macro"))
print('precision_score: ', precision_score(y_test, predicted, average="macro"))
print('recall_score', recall_score(y_test, predicted, average="macro")) 

# calculating the main quality metrics
print(metrics.classification_report(y_test, predicted))

f1_score:  0.7178962381300287
precision_score:  0.7427915342771418
recall_score 0.7171906782701403
              precision    recall  f1-score   support

          -2       0.89      0.83      0.86       987
          -1       0.45      0.23      0.30      1278
           0       0.56      0.77      0.65      1969
           1       0.89      0.75      0.81      1023
           2       0.94      1.00      0.97       996

    accuracy                           0.70      6253
   macro avg       0.74      0.72      0.72      6253
weighted avg       0.70      0.70      0.69      6253



На тестовых:

In [35]:
X=np.stack(df_test['original_data'])
y=df_test['score']
predicted = model.predict(X)
predicted-=2
print('f1_score: ', f1_score(y, predicted, average="macro"))
print('precision_score: ', precision_score(y, predicted, average="macro"))
print('recall_score', recall_score(y, predicted, average="macro")) 

# calculating the main quality metrics
print(metrics.classification_report(y, predicted))

f1_score:  0.6299685319504189
precision_score:  0.6284077307343471
recall_score 0.66069059860673
              precision    recall  f1-score   support

          -2       0.60      0.62      0.61       473
          -1       0.75      0.47      0.58      2747
           0       0.69      0.85      0.77      4199
           1       0.59      0.57      0.58       524
           2       0.50      0.79      0.61       105

    accuracy                           0.69      8048
   macro avg       0.63      0.66      0.63      8048
weighted avg       0.70      0.69      0.68      8048



## AdaBoost

In [36]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X_train, y_train_pos)

AdaBoostClassifier()

In [37]:
predicted = model.predict(X_test)-2
print('f1_score: ', f1_score(y_test, predicted, average="macro"))
print('precision_score: ', precision_score(y_test, predicted, average="macro"))
print('recall_score', recall_score(y_test, predicted, average="macro")) 

# calculating the main quality metrics
print(metrics.classification_report(y_test, predicted))

f1_score:  0.1973367702889972
precision_score:  0.25791616481539303
recall_score 0.26464193773057126
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00       987
          -1       0.20      0.00      0.00      1278
           0       0.33      0.90      0.49      1969
           1       0.26      0.04      0.07      1023
           2       0.49      0.38      0.43       996

    accuracy                           0.35      6253
   macro avg       0.26      0.26      0.20      6253
weighted avg       0.27      0.35      0.23      6253



## LSTM с эмбедингами из navec

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.models import Sequential

In [39]:
!pip install slovnet
from slovnet.model.emb import NavecEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.0 MB/s eta 0:00:00


In [46]:
xtrain, xtest, ytrain, ytest = train_test_split(df_train['original_data'], df_train['score'], shuffle=True, test_size=0.2)
# find the length of the largest sentence in training data
max_len = xtrain.apply(lambda x: len(x)).max()
print(f'Max number of words in a text in training data: {max_len}')

Max number of words in a text in training data: 5984


In [48]:
max_words = 10000
tokenizer = text.Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(xtrain)
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
word_index = tokenizer.word_index

print('text example:', xtrain[0])
print('sequence of indices(before padding):', xtrain_seq[0])
print('sequence of indices(after padding):', xtrain_pad[0])

text example: гала небольшой коментарий pocahontasm рекомендовать прочитать ваш рекомендация зайти форум дежурить почитать вспомнить изречение конфуций очень трудно искать чёрный кошка тёмный комната особенность нех парень который спорить пурга голова переубеждать совершенно толк скорее нужно высылать целый набор ссылка сначала перечитать писать пресса это выстроить линейка публикация возникать логический связь сколько фильтровать информация явно уметь совершенно чётко пролёживаться компост который нашпиговать средство массовый информация россия
sequence of indices(before padding): [1608, 537, 76, 118, 585, 101, 1835, 920, 1044, 16, 376, 18, 2043, 607, 6302, 8686, 11, 5695, 36, 69, 6681, 4172, 1439, 36, 676, 9, 2648, 13, 522, 5981, 8066, 279, 1, 1044, 118, 3535, 16, 805, 169, 69, 7106, 36, 80, 369, 440, 36, 805, 367, 1608, 118]
sequence of indices(after padding): [   0    0    0 ...  367 1608  118]


In [49]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-03-24 09:47:59--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  43.1MB/s    in 1.2s    

2023-03-24 09:48:00 (43.1 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [50]:
from navec import Navec
path="navec_hudlit_v1_12B_500K_300d_100q.tar"
navec = Navec.load(path)
navec['навек'].shape
embedding_vectors=navec

In [51]:
emb_dim = 300
if max_words is not None: 
    vocab_len = max_words 
else:
    vocab_len = len(word_index)+1
embedding_matrix = np.zeros((vocab_len, emb_dim))
oov_count = 0
oov_words = []
for word, idx in word_index.items():
    if idx < vocab_len:
        embedding_vector = embedding_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
        else:
            oov_count += 1 
            oov_words.append(word)

print(f'Some out of valubulary words: {oov_words[0:5]}')

Some out of valubulary words: ['чтоть', 'ктоть', 'какойтый', 'гдеть', 'почемуть']


In [52]:
print(f'{oov_count} out of {vocab_len} words were OOV.')

447 out of 10000 words were OOV.


In [54]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_len, emb_dim, trainable = False, weights=[embedding_matrix]))
lstm_model.add(LSTM(128, return_sequences=False))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation = 'sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         3000000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               219648    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3,219,777
Trainable params: 219,777
Non-trainable params: 3,000,000
_________________________________________________________________
None


In [55]:
%%time
batch_size = 256
epochs  = 10
history = lstm_model.fit(xtrain_pad, np.asarray(ytrain), validation_data=(xtest_pad, np.asarray(ytest)), batch_size = batch_size, epochs = epochs)

Epoch 1/10
59/59 [==============================] - 58s 876ms/step - loss: -5.0829 - accuracy: 0.5148 - val_loss: -7.9121 - val_accuracy: 0.5314
Epoch 2/10
59/59 [==============================] - 51s 866ms/step - loss: -9.4368 - accuracy: 0.5191 - val_loss: -10.9884 - val_accuracy: 0.5314
Epoch 3/10
59/59 [==============================] - 51s 866ms/step - loss: -12.4254 - accuracy: 0.5191 - val_loss: -13.8943 - val_accuracy: 0.5314
Epoch 4/10
59/59 [==============================] - 49s 839ms/step - loss: -15.2757 - accuracy: 0.5191 - val_loss: -16.7579 - val_accuracy: 0.5314
Epoch 5/10
59/59 [==============================] - 50s 841ms/step - loss: -18.1650 - accuracy: 0.5191 - val_loss: -19.6074 - val_accuracy: 0.5314
Epoch 6/10
59/59 [==============================] - 51s 866ms/step - loss: -21.0097 - accuracy: 0.5191 - val_loss: -22.4116 - val_accuracy: 0.5314
Epoch 7/10
59/59 [==============================] - 49s 838ms/step - loss: -23.7470 - accuracy: 0.5191 - val_loss: -25.20

In [56]:
test_lstm_results = lstm_model.evaluate(xtest_pad, np.asarray(ytest), verbose=0, batch_size=256)
print(f'Test accuracy: {test_lstm_results[1]*100:0.2f}')

Test accuracy: 53.14


In [61]:
print(df_test["original_data"])

0       [-0.0007005358, -0.0010926597, -0.0008658411, ...
1       [0.00050507725, 0.0015609633, -0.00063179614, ...
2       [0.0012362582, -0.0012174068, 0.001658022, -0....
3       [-0.0006065422, 0.0012920747, 0.0011434853, 0....
4       [-3.506144e-06, -0.0011744092, 0.0005202212, 0...
                              ...                        
8043    [0.00033375024, -4.824559e-05, 0.0011086279, 0...
8044    [-0.0002488044, -0.0012059625, -0.0004676775, ...
8045    [-0.00030789195, 0.0008987969, 0.0014729772, -...
8046    [-0.0011191082, 0.0011925958, -0.0012218261, 0...
8047    [-0.0016263147, -0.0007113282, -0.001607763, 0...
Name: original_data, Length: 8048, dtype: object


In [62]:
test_data = "/kaggle/input/nlp-data/test_data_exam_1.xlsx"
df_test = pd.read_excel(test_data, 0) 
df_test=clear_and_lematization(df_test)
df_test=stop_words(df_test)
x_test=df_test["original_data"]
# create the vocabulary by fitting on x_train text
tokenizer.fit_on_texts(x_test)
# generate the sequence of tokens
xtrain_seq = tokenizer.texts_to_sequences(x_test)

# pad the sequences
x_test = sequence.pad_sequences(xtrain_seq, maxlen=max_len)

predicted=lstm_model.predict(x_test)
print(metrics.classification_report(df_test["score"], predicted))

  0%|          | 0/8048 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
  0%|          | 0/8048 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|██████████| 8048/8048 [00:09<00:00, 850.81it/s]


252/252 [==============================] - 34s 134ms/step
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00       473
          -1       0.00      0.00      0.00      2747
           0       0.52      1.00      0.69      4199
           1       0.00      0.00      0.00       524
           2       0.00      0.00      0.00       105

    accuracy                           0.52      8048
   macro avg       0.10      0.20      0.14      8048
weighted avg       0.27      0.52      0.36      8048



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
